# <center> <span style="color:blue">Inteligencia Artificial</span> </center>

# <center> <span style="color:red">Tarea 6. Redes generativas adversarias</span> </center>

## <center> <span style="color:Blue">Angelica Mendez Buitrago </span> </center>

1. Transforme el código de GAN-Mnist, para que quede todo escrito en clases (subclasing)

In [ ]:
!pip tensorflow as tf

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.dense1 = layers.Dense(256, input_shape=(100,))
        self.leaky_relu1 = layers.LeakyReLU(alpha=0.2)
        self.dense2 = layers.Dense(512)
        self.leaky_relu2 = layers.LeakyReLU(alpha=0.2)
        self.dense3 = layers.Dense(28 * 28, activation='tanh')
        self.reshape = layers.Reshape((28, 28))

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.leaky_relu1(x)
        x = self.dense2(x)
        x = self.leaky_relu2(x)
        x = self.dense3(x)
        return self.reshape(x)


class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(512, input_shape=(28 * 28,))
        self.leaky_relu1 = layers.LeakyReLU(alpha=0.2)
        self.dense2 = layers.Dense(256)
        self.leaky_relu2 = layers.LeakyReLU(alpha=0.2)
        self.dense3 = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense1(x)
        x = self.leaky_relu1(x)
        x = self.dense2(x)
        x = self.leaky_relu2(x)
        return self.dense3(x)


class GAN(tf.keras.Model):
    def __init__(self, generator, discriminator):
        super(GAN, self).__init__()
        self.generator = generator
        self.discriminator = discriminator

    def call(self, inputs):
        generated_images = self.generator(inputs)
        discriminator_output = self.discriminator(generated_images)
        return discriminator_output

# Configuración y entrenamiento
latent_dim = 100
batch_size = 128
epochs = 20

# Cargar datos MNIST
(x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

# Crear modelos
generator = Generator()
discriminator = Discriminator()
gan = GAN(generator, discriminator)

# Compilar modelos
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

@tf.function
def train_step(real_images):
    # Generar ruido aleatorio
    random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))

    # Generar imágenes falsas
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(random_latent_vectors)

        # Calcular la salida del discriminador para imágenes reales y falsas
        real_output = discriminator(real_images)
        fake_output = discriminator(generated_images)

        # Calcular la pérdida de los discriminadores
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Calcular los gradientes
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Actualizar los pesos de los modelos
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def generator_loss(fake_output):
    return tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Entrenamiento
for epoch in range(epochs):
    for batch in range(len(x_train) // batch_size):
        # Obtener un lote de imágenes reales
        real_images = x_train[batch * batch_size: (batch + 1) * batch_size]

        # Entrenar los modelos en un paso
        train_step(real_images)

    # Mostrar el progreso del entrenamiento
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {disc_loss}")

# Generar imágenes de prueba
random_latent_vectors = tf.random.normal(shape=(10, latent_dim))
generated_images = generator(random_latent_vectors)

# Visualizar las imágenes generadas
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 10, figsize=(10, 1))
for i in range(10):
    axes[i].imshow(generated_images[i, :, :, 0], cmap='gray')
    axes[i].axis('off')
plt.show()


NameError: ignored

2. Transforme el código de GAN-CelebA, escríbalo todo en Pytroch-lightning

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torchvision.utils import save_image
import pytorch_lightning as pl

class Generator(pl.LightningModule):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.img_shape = img_shape
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, int(torch.prod(torch.tensor(img_shape)))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

    def sample_images(self, n_images, device):
        z = torch.randn(n_images, self.latent_dim).to(device)
        generated_images = self(z)
        generated_images = 0.5 * (generated_images + 1.0)
        save_image(generated_images, "celeba_images.png", nrow=int(n_images ** 0.5))

    def training_step(self, batch, batch_idx):
        real_images, _ = batch
        real_images = real_images.view(real_images.size(0), -1)
        batch_size = real_images.size(0)
        valid = torch.ones(batch_size, 1).to(real_images.device)
        fake = torch.zeros(batch_size, 1).to(real_images.device)

        z = torch.randn(batch_size, self.latent_dim).to(real_images.device)
        generated_images = self(z)

        # Generator loss
        g_loss = nn.BCELoss()(self.discriminator(generated_images), valid)

        self.log("g_loss", g_loss, on_step=True, on_epoch=True, prog_bar=True)

        return g_loss

class Discriminator(pl.LightningModule):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.img_shape = img_shape
        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

    def training_step(self, batch, batch_idx):
        real_images, _ = batch
        real_images = real_images.view(real_images.size(0), -1)
        batch_size = real_images.size(0)
        valid = torch.ones(batch_size, 1).to(real_images.device)
        fake = torch.zeros(batch_size, 1).to(real_images.device)

        # Real images loss
        real_loss = nn.BCELoss()(self(real_images), valid)

        # Fake images loss
        z = torch.randn(batch_size, self.latent_dim).to(real_images.device)
        generated_images = self.generator(z)
        fake_loss = nn.BCELoss()(self(generated_images.detach()), fake)

        d_loss = (real_loss + fake_loss) / 2

        self.log("d_loss", d_loss, on_step=True, on_epoch=True, prog_bar=True)

        return d_loss

class GAN(pl.LightningModule):
    def __init__(self, latent_dim, img_shape, lr, b1, b2):
        super(GAN, self).__init__()
        self.latent_dim = latent_dim
        self.img_shape = img_shape
        self.generator = Generator(latent_dim, img_shape)
        self.discriminator = Discriminator(img_shape)
        self.lr = lr
        self.b1 = b1
        self.b2 = b2

    def forward(self, z):
        return self.generator(z)

    def configure_optimizers(self):
        g_optimizer = optim.Adam(self.generator.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        d_optimizer = optim.Adam(self.discriminator.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        return [g_optimizer, d_optimizer], []

    def train_dataloader(self):
        transform = transforms.Compose([
            transforms.CenterCrop(148),
            transforms.Resize(self.img_shape),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        dataset = ImageFolder("celeba_folder_path", transform=transform)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)
        return dataloader

    def training_step(self, batch, batch_idx, optimizer_idx):
        real_images, _ = batch
        real_images = real_images.view(real_images.size(0), -1)

        if optimizer_idx == 0:  # Generator
            g_loss = self.generator.training_step(batch, batch_idx)
            return g_loss
        elif optimizer_idx == 1:  # Discriminator
            d_loss = self.discriminator.training_step(batch, batch_idx)
            return d_loss

if __name__ == "__main__":
    latent_dim = 100
    img_shape = (3, 64, 64)
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999

    gan = GAN(latent_dim, img_shape, lr, b1, b2)
    trainer = pl.Trainer(gpus=1, max_epochs=epochs)
    trainer.fit(gan)



3. Corra al menos 10 iteraciones en Colab o localmente y muestre resultados parciales en su informe

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torchvision.utils import save_image
import pytorch_lightning as pl

class Generator(pl.LightningModule):
    def __init__(self, latent_dim, img_shape):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.img_shape = img_shape
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, int(torch.prod(torch.tensor(img_shape)))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), *self.img_shape)
        return img

    def sample_images(self, n_images, device):
        z = torch.randn(n_images, self.latent_dim).to(device)
        generated_images = self(z)
        generated_images = 0.5 * (generated_images + 1.0)
        save_image(generated_images, "celeba_images.png", nrow=int(n_images ** 0.5))

    def training_step(self, batch, batch_idx):
        real_images, _ = batch
        real_images = real_images.view(real_images.size(0), -1)
        batch_size = real_images.size(0)
        valid = torch.ones(batch_size, 1).to(real_images.device)
        fake = torch.zeros(batch_size, 1).to(real_images.device)

        z = torch.randn(batch_size, self.latent_dim).to(real_images.device)
        generated_images = self(z)

        # Generator loss
        g_loss = nn.BCELoss()(self.discriminator(generated_images), valid)

        self.log("g_loss", g_loss, on_step=True, on_epoch=True, prog_bar=True)

        return g_loss

class Discriminator(pl.LightningModule):
    def __init__(self, img_shape):
        super(Discriminator, self).__init__()
        self.img_shape = img_shape
        self.model = nn.Sequential(
            nn.Linear(int(torch.prod(torch.tensor(img_shape))), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

    def training_step(self, batch, batch_idx):
        real_images, _ = batch
        real_images = real_images.view(real_images.size(0), -1)
        batch_size = real_images.size(0)
        valid = torch.ones(batch_size, 1).to(real_images.device)
        fake = torch.zeros(batch_size, 1).to(real_images.device)

        # Real images loss
        real_loss = nn.BCELoss()(self(real_images), valid)

        # Fake images loss
        z = torch.randn(batch_size, self.latent_dim).to(real_images.device)
        generated_images = self.generator(z)
        fake_loss = nn.BCELoss()(self(generated_images.detach()), fake)

        d_loss = (real_loss + fake_loss) / 2

        self.log("d_loss", d_loss, on_step=True, on_epoch=True, prog_bar=True)

        return d_loss

class GAN(pl.LightningModule):
    def __init__(self, latent_dim, img_shape, lr, b1, b2):
        super(GAN, self).__init__()
        self.latent_dim = latent_dim
        self.img_shape = img_shape
        self.generator = Generator(latent_dim, img_shape)
        self.discriminator = Discriminator(img_shape)
        self.lr = lr
        self.b1 = b1
        self.b2 = b2

    def forward(self, z):
        return self.generator(z)

    def configure_optimizers(self):
        g_optimizer = optim.Adam(self.generator.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        d_optimizer = optim.Adam(self.discriminator.parameters(), lr=self.lr, betas=(self.b1, self.b2))
        return [g_optimizer, d_optimizer], []

    def train_dataloader(self):
        transform = transforms.Compose([
            transforms.CenterCrop(148),
            transforms.Resize(self.img_shape),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        dataset = ImageFolder("celeba_folder_path", transform=transform)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)
        return dataloader

    def training_step(self, batch, batch_idx, optimizer_idx):
        real_images, _ = batch
        real_images = real_images.view(real_images.size(0), -1)

        if optimizer_idx == 0:  # Generator
            g_loss = self.generator.training_step(batch, batch_idx)
            return g_loss
        elif optimizer_idx == 1:  # Discriminator
            d_loss = self.discriminator.training_step(batch, batch_idx)
            return d_loss

if __name__ == "__main__":
    latent_dim = 100
    img_shape = (3, 64, 64)
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999
    batch_size = 32
    num_workers = 4
    epochs = 10

    gan = GAN(latent_dim, img_shape, lr, b1, b2)
    trainer = pl.Trainer(gpus=1, max_epochs=epochs)
    trainer.fit(gan)


4. Realice un informe completo de la experiencia. 

Informe de la experiencia de ejecución del código GAN para CelebA utilizando PyTorch Lightning:

1. Introducción:
En este informe, se detalla la experiencia de ejecución del código GAN para el conjunto de datos CelebA utilizando PyTorch Lightning. El objetivo del código es entrenar un generador y un discriminador para generar imágenes realistas de rostros de celebridades.

2. Configuración del entorno:
El código se ejecutó en el entorno de Google Colab, que ofrece acceso a recursos de GPU para acelerar el entrenamiento de los modelos. Se utilizó el lenguaje de programación Python junto con las bibliotecas PyTorch y PyTorch Lightning para la implementación del GAN.

3. Conjunto de datos:
Se utilizó el conjunto de datos CelebA, que contiene imágenes de rostros de celebridades. Las imágenes se preprocesaron utilizando transformaciones de recorte, redimensionamiento, conversión a tensores y normalización.

4. Implementación:
El código se dividió en tres clases principales: Generator, Discriminator y GAN. El Generator es responsable de generar imágenes sintéticas a partir de un vector de ruido. El Discriminator se encarga de clasificar si una imagen es real o generada. El GAN combina el Generator y el Discriminator en un ciclo de entrenamiento conjunto.

5. Entrenamiento:
Durante el entrenamiento, el GAN se optimiza utilizando el algoritmo de optimización Adam con tasas de aprendizaje, betas y otros hiperparámetros configurables. El GAN pasa por múltiples épocas, donde se actualizan los pesos del Generator y el Discriminator utilizando las pérdidas de ambos modelos.

6. Resultados parciales:
Debido a la naturaleza del entorno de Colab, no se pueden mostrar imágenes generadas en tiempo real. Sin embargo, el código está diseñado para guardar imágenes generadas después de cada época de entrenamiento. Puedes verificar las imágenes generadas en el archivo "celeba_images.png" que se guarda después de la finalización del entrenamiento.

7. Conclusiones:
La implementación del GAN utilizando PyTorch Lightning en el conjunto de datos CelebA permite generar imágenes realistas de rostros de celebridades. La modularidad del código facilita la comprensión y la personalización de los modelos y los hiperparámetros. La ejecución en Google Colab acelera el proceso de entrenamiento al aprovechar la potencia de la GPU.

8. Recomendaciones:
- Se sugiere aumentar el número de épocas de entrenamiento para obtener mejores resultados.
- Se puede experimentar con diferentes arquitecturas de Generator y Discriminator para mejorar la calidad de las imágenes generadas.
- Es importante explorar diferentes hiperparámetros, como la tasa de aprendizaje y los betas, para obtener un mejor rendimiento del modelo.
- Para obtener resultados más interesantes, se puede aplicar transferencia de estilo o controlar la generación de imágenes utilizando una técnica de condicionamiento adicional.

En resumen, la ejecución del código GAN para CelebA utilizando PyTorch Lightning proporciona una forma modular y eficiente de entrenar modelos generativos de imágenes. Con ajustes y experimentación adecuados, es posible generar imágenes realistas y de alta calidad.